In [ ]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter
from solve_network import palette

In [ ]:
if __name__ == "__main__":
    # Detect running outside of snakemake and mock snakemake for testing
    if "snakemake" not in globals():
        from _helpers import mock_snakemake

        snakemake = mock_snakemake(
            "plot_summary", year="2030", zone="IE", palette="p3", policy="cfe100"
        )

    config = snakemake.config
    scaling = int(config["time_sampling"][0])  # temporal scaling -- 3/1 for 3H/1H

    # Wildcards & Settings
    policy = snakemake.wildcards.policy[:3]
    penetration = float(snakemake.wildcards.policy[3:]) / 100 if policy != "ref" else 0
    tech_palette = snakemake.wildcards.palette
    zone = snakemake.wildcards.zone
    year = snakemake.wildcards.year

    datacenters = snakemake.config["ci"]["datacenters"]
    locations = list(datacenters.keys())
    names = list(datacenters.values())
    flexibilities = snakemake.config["ci"]["flexibility"]

    # techs for CFE hourly matching, extracted from palette
    palette_techs = palette(tech_palette)

    (
        clean_techs,
        storage_techs,
        storage_charge_techs,
        storage_discharge_techs,
    ) = palette_techs

    # renaming technologies for plotting
    clean_chargers = [tech.replace(" ", "_") for tech in storage_charge_techs]
    clean_dischargers = [tech.replace(" ", "_") for tech in storage_discharge_techs]

    def tech_names(base_names, year):
        return [f"{name.replace(' ', '_')}-{year}" for name in base_names]

    # expected technology names with year
    exp_generators = tech_names(["offwind-ac", "offwind-dc", "onwind", "solar"], year)
    exp_links = tech_names(["OCGT"], year)
    exp_chargers = tech_names(["battery charger", "H2 Electrolysis"], year)
    exp_dischargers = tech_names(["battery discharger", "H2 Fuel Cell"], year)

    # Assign colors
    tech_colors = snakemake.config["tech_colors"]

    # Rename mappings
    rename_ci_cost = pd.Series(
        {
            "onwind": "onshore wind",
            "solar": "solar",
            "grid": "grid imports",
            "revenue": "revenue",
            "battery_storage": "battery",
            "battery_inverter": "battery",
            "battery_discharger": "battery",
            "hydrogen_storage": "hydrogen storage",
            "hydrogen_electrolysis": "hydrogen storage",
            "hydrogen_fuel_cell": "hydrogen storage",
            "adv_geothermal": "advanced dispatchable",
            "allam_ccs": "NG-Allam",
        }
    )

    rename_ci_capacity = pd.Series(
        {
            "onwind": "onshore wind",
            "solar": "solar",
            "battery_discharger": "battery",
            "H2_Fuel_Cell": "hydrogen fuel cell",
            "H2_Electrolysis": "hydrogen electrolysis",
            "adv_geothermal": "advanced dispatchable",
            "allam_ccs": "NG-Allam",
        }
    )

    rename_scen = {
        "0": "0%",
        "10": "10%",
        "25": "25%",
    }

    preferred_order = pd.Index(
        [
            "advanced dispatchable",
            "NG-Allam",
            "Gas OC",
            "offshore wind",
            "onshore wind",
            "solar",
            "battery",
            "hydrogen storage",
            "hydrogen electrolysis",
            "hydrogen fuel cell",
        ]
    )

## Data Preparation

In [ ]:
df = pd.read_csv(snakemake.input.summary, index_col=0, header=[0, 1])

In [ ]:
ldf = pd.concat(
    [
        df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_techs]].rename(
            {"ci_cap_" + t: t for t in clean_techs}
        ),
        df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_dischargers]].rename(
            {"ci_cap_" + t: t for t in clean_dischargers}
        ),
        df.loc[["ci_cap_" + t.replace(" ", "_") for t in clean_chargers]]
        .rename({"ci_cap_" + t: t for t in clean_chargers})
        .drop(["battery_charger"]),
    ]
)

# Drop rows with all values less than 0.1
to_drop = ldf.index[(ldf < 0.1).all(axis=1)]
ldf.drop(to_drop, inplace=True)

# Rename columns and indices
ldf.rename(columns=rename_scen, level=0, inplace=True)
ldf.rename(index=rename_ci_capacity, level=0, inplace=True)

# Reorder and sort the final DataFrame
new_index = preferred_order.intersection(ldf.index).append(
    ldf.index.difference(preferred_order)
)
ldf = ldf.loc[new_index].sort_index(
    axis="columns", level=[1, 0], ascending=[False, True]
)

In [ ]:
ldf

In [ ]:
E = ldf.xs("NG-Allam").xs("IE5 0", level=1)
type(E)
E

# Learning curve

In [ ]:
def experience_curve(E, C0, E0, LR):
    """
    Calculate the investment cost c as a function of experience E.

    Parameters (all float):
    E: Invested capacity (updated experience) in MW.
    C0: The initial investment costs when experience E is E0 in EUR/MW.
    E0: Initial capacity (initial experience) in MW.
    LR: The learning rate, i.e., the percentage cost reduction for each doubling of cumulative experience.

    Returns (also float):
    alpha: The experience exponent
    c(E): The investment cost c for the given capacity (E + E0).
    """

    alpha = np.log2(1 / (1 - LR))

    c = C0 * ((E + E0) / E0) ** (-alpha)

    return round(alpha, 3), c

In [ ]:
C0 = 2500  # EUR/kW
E0 = 500  # MW
LR = 0.2  # 20%

### (sketch) Results for CFE score = 100% and C&I consumers in Ireland 

In [ ]:
E

In [ ]:
experience_curve(E, C0=2500, E0=500, LR=0.2)

In [ ]:
experience_curve(E, C0=2500, E0=1000, LR=0.2)

In [ ]:
experience_curve(E, C0=2500, E0=1000, LR=0.1)

### (sketch) Results for CFE score = 100% and C&I consumers in Germany 

In [ ]:
E_germany = E * 38410 / 2200
E_germany

In [ ]:
experience_curve(E=E_germany, C0=2500, E0=500, LR=0.2)

In [ ]:
experience_curve(E=E_germany, C0=2500, E0=1000, LR=0.2)

In [ ]:
experience_curve(E=E_germany, C0=2500, E0=1000, LR=0.1)

In [ ]:
# to get CCGT parity
experience_curve(E=2e4, C0=2500, E0=500, LR=0.2)